In [ ]:
library(hh.analytics)
library(hh.snowflake)

load.packages()

In [ ]:
start = querySnowflake("select max(response_id) as MAX_RESPONSE_ID from edwprodhh.edi_837_parser.response") %>% pull(max_response_id)

In [ ]:
list.files(
    "\\\\hh-fileserver01\\TempUL2\\IU_Health_Complex\\837_FILES_IN\\2025",
    full.names = TRUE
) %>% 
    data.frame(
        file_path = .
    ) %>% 
    filter(
        !file_path %>% str_detect("xl212927fw_250426_i5") &
        file_path %>% str_detect("837$")
    ) %>% 
    mutate(
        response_id = row_number() + start,
    ) %>% 
    {
        df_ = .

        for (i in 1:nrow(df_)) {

            df_ %>% 
                slice(i) %>% 
                transmute(
                    response_id,
                    response_body = file_path %>% map(~ .x %>% readLines() %>% paste0(collapse = "\\n")) %>% as.character(),
                    file_name = file_path %>% str_extract("[^/]*$")
                ) %>%
                insertValuesSnowflakeBatch(
                    "edwprodhh.edi_837_parser.response"
                )

        }

    }